In [1]:
import numpy as np
import math
from pathlib import Path

import rasterio
from rasterio.plot import show_hist
from rasterio.dtypes import int16

In [2]:
@np.vectorize
def is_prime(n):
    if n % 2 == 0 and n > 2:
        return False
    return all(n % i for i in range(3, int(np.sqrt(n)) + 1, 2))

@np.vectorize
def is_polygonal(s, x):
    assert s > 2 and s % 1 == 0 and x % 1 == 0
    n = (np.sqrt(8 * (s - 2) * x + (s - 4) ** 2) + (s - 4)) / (2 * (s - 2))
    return n % 1 == 0

@np.vectorize
def is_fibonacci(n):
    a, b = 0,1
    while a < n:
        a, b = b, a + b
    return a == n

@np.vectorize
def is_perfect(n):
    sum = 1
    i = 2
    while i * i <= n:
        if n % i == 0:
            sum = sum + i + n/i
        i += 1
    return sum == n and n != 1

def is_triangular(n):
    return is_polygonal(3, n)
    
def is_rectangular(n):
    return is_polygonal(4, n)

def is_pentagonal(n):
    return is_polygonal(5, n)

def is_hexagonal(n):
    return is_polygonal(6, n)

# Calculate whether each value in each band satisfies a function
# Then count the number of times it satisfies the function, and write that out

funcs = [
    is_prime,
    is_triangular, is_rectangular, is_pentagonal, is_hexagonal,
    is_fibonacci,
    is_perfect
]

def compute(input_file, dtype=int16, scale=100, band_limit=None):
    with rasterio.open(input_file, 'r') as src:
        shape = (src.indexes[-1], *src.shape)
        for func in funcs:
            src_profile = src.profile.copy()
            output = np.zeros(shape, dtype)
            for band in src.indexes[:band_limit]:
                data = src.read(band)
                data *= scale
                output[band-1] = func(data.astype(dtype))
    
            src_profile.update(count=1)
            src_profile.update(dtype=dtype)
            count_satisfaction = np.apply_along_axis(np.sum, 0, output)
            with rasterio.open(Path(f'./{input_file.stem}-{func.__name__}-{(band_limit or len(src.indexes))}.tif'), 'w', **src_profile) as dst:
                dst.write(count_satisfaction, 1)

In [3]:
%%time
compute(Path('./continuous.tif'), band_limit=10)

C:\Users\Tigeir\AppData\Local\Temp\ipykernel_24164\2840232669.py:60: RuntimeWarning: overflow encountered in multiply
  data *= scale
C:\Users\Tigeir\AppData\Local\Temp\ipykernel_24164\2840232669.py:61: RuntimeWarning: invalid value encountered in cast
  output[band-1] = func(data.astype(dtype))


CPU times: total: 2.16 s
Wall time: 9.52 s


In [4]:
%%time
compute(Path('./continuous.tif'), band_limit=100)

C:\Users\Tigeir\AppData\Local\Temp\ipykernel_24164\2840232669.py:60: RuntimeWarning: overflow encountered in multiply
  data *= scale
C:\Users\Tigeir\AppData\Local\Temp\ipykernel_24164\2840232669.py:61: RuntimeWarning: invalid value encountered in cast
  output[band-1] = func(data.astype(dtype))


CPU times: total: 4.14 s
Wall time: 34.4 s


In [5]:
%%time
compute(Path('./continuous.tif'), band_limit=1000)

C:\Users\Tigeir\AppData\Local\Temp\ipykernel_24164\2840232669.py:60: RuntimeWarning: overflow encountered in multiply
  data *= scale
C:\Users\Tigeir\AppData\Local\Temp\ipykernel_24164\2840232669.py:61: RuntimeWarning: invalid value encountered in cast
  output[band-1] = func(data.astype(dtype))


CPU times: total: 48.2 s
Wall time: 4min 49s


In [6]:
%%time
compute(Path('./continuous.tif'), band_limit=10000)

C:\Users\Tigeir\AppData\Local\Temp\ipykernel_24164\2840232669.py:60: RuntimeWarning: overflow encountered in multiply
  data *= scale
C:\Users\Tigeir\AppData\Local\Temp\ipykernel_24164\2840232669.py:61: RuntimeWarning: invalid value encountered in cast
  output[band-1] = func(data.astype(dtype))


CPU times: total: 7min 46s
Wall time: 48min 56s


In [7]:
%%time
compute(Path('./discrete.tif'), scale=1, band_limit=10)

CPU times: total: 1.25 s
Wall time: 8.73 s


In [8]:
%%time
compute(Path('./discrete.tif'), scale=1, band_limit=100)

CPU times: total: 3 s
Wall time: 30.5 s


In [9]:
%%time
compute(Path('./discrete.tif'), scale=1, band_limit=1000)

CPU times: total: 22.7 s
Wall time: 4min 8s


In [10]:
%%time
compute(Path('./discrete.tif'), scale=1, band_limit=10000)

CPU times: total: 4min 19s
Wall time: 41min 46s
